In [7]:
import numpy as np
import struct
import sys
import os
from vec_io import get_zeroled_names, read_vec
from vis_io import get_vtr, write_vtr, get_mesh
import vtk
from vtkmodules.util import numpy_support

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
vec_dir = 'data'
vec_paths = sorted([os.path.join(vec_dir, vec_file) for vec_file in os.listdir(vec_dir)])
vec0 = read_vec(vec_paths[0])

In [34]:
# generate seeds in every on each grid point of (48,48,48)
dim_coord = np.arange(48., dtype=np.float32)
mesh = get_mesh(dim_coord,dim_coord,dim_coord)
mesh = mesh.reshape(-1, 3)
seeds = vtk.vtkPoints()
seeds.SetData(numpy_support.numpy_to_vtk(mesh))
seeds_pd = vtk.vtkPolyData()
seeds_pd.SetPoints(seeds)
# seeds_dataset = vtk.vtkPointSet()
# seeds_dataset.SetPoints(seeds)
# seeds_source = vtk.vtkPointSource()
# seeds_source.SetInputData(seeds_dataset)

# get vector field in vtk format
dim_coord = np.arange(48., dtype=np.float32)
vec_vtr = get_vtr(vec0.shape[:-1], dim_coord, dim_coord, dim_coord, {'velocity': vec0.reshape(-1, 3)})

meshe generated: (48, 48, 48, 3)


In [35]:
# get stream tracer
tem_speed =0.0001
max_error = 1e-12
min_intstepsize = 0.01
max_intstepsize = 0.5
max_steps = 1000
max_length = 47


st = vtk.vtkStreamTracer()

st.SetInputData(vec_vtr)
st.SetSourceData(seeds_pd)

integrator = vtk.vtkRungeKutta45()
st.SetIntegratorTypeToRungeKutta45()
st.SetIntegrationDirectionToForward()
st.SetInitialIntegrationStep(0.2)
st.SetMinimumIntegrationStep(min_intstepsize)
st.SetMaximumIntegrationStep(max_intstepsize)
st.SetMaximumNumberOfSteps(max_steps)
st.SetMaximumPropagation(max_length)
st.SetTerminalSpeed(tem_speed)

# st.SetSourceData(seeds_dataset)

st.Update()

pdm = st.GetOutput()
pdm.GetNumberOfLines()

0

In [38]:
pdm.GetNumberOfCells()

0

In [36]:
pdm.BuildCells()

In [24]:
st.SetSource()

AttributeError: 'vtkmodules.vtkFiltersFlowPaths.vtkStreamTracer' object has no attribute 'SetSource'

In [42]:
pd = st.GetSource()

In [ ]:
pd.

In [26]:
pointdata = vec_vtr.GetPointData()